Setup (general)

In [1]:
%load_ext autoreload
%autoreload 2

Load the toolkit, setup input data


In [2]:
import spikeextractors as se
import spiketoolkit as st
import spikesorters as ss

# filepath = 'G:/Data_julien/Klusta_results/Barrel1/Barrel1.dat'
filepath = 'C:/tmp/TT_Hpc5/TT_Hpc5.dat'
rec = se.BinDatRecordingExtractor(filepath,
                                  numchan=4,
                                  dtype='int16',
                                  sampling_frequency=32000)

rec = rec.load_probe_file('C:/tmp/TT_Hpc5/TT_Hpc5.prb')
# rec = rec.load_probe_file('G:/Data_julien/Klusta_results/Barrel1/Barrel1.prb')

10:35:45 [I] klustakwik KlustaKwik2 version 0.2.6


In [3]:
print(ss.available_sorters())
print(ss.installed_sorter_list)
print(ss.get_default_params('klusta'))

klusta_params = ss.get_default_params('klusta')
# sorting_klusta = ss.run_klusta(recording=rec,
#                                verbose=True,
#                                **klusta_params)

['hdsort', 'herdingspikes', 'ironclust', 'kilosort', 'kilosort2', 'klusta', 'mountainsort4', 'spykingcircus', 'tridesclous', 'waveclus']
[<class 'spikesorters.klusta.klusta.KlustaSorter'>]
{'adjacency_radius': None, 'threshold_strong_std_factor': 5, 'threshold_weak_std_factor': 2, 'detect_sign': -1, 'extract_s_before': 16, 'extract_s_after': 32, 'n_features_per_channel': 3, 'pca_n_waveforms_max': 10000, 'num_starting_clusters': 50}


In [4]:
# st.postprocessing.export_to_phy(
#     recording=rec,
#     sorting=sorting_klusta,
#     output_folder='G:/Data_julien/Klusta_results/Barrel1/phy'
# )

In [5]:
# sor = se.PhySortingExtractor(
#     'G:/Data_julien/Klusta_results/Barrel1/phy'
# )

sor = se.KlustaSortingExtractor(file_or_folder_path='C:/tmp/TT_Hpc5')
uids = sor.get_unit_ids()
good_units = [uid for uid in uids if sor.get_unit_property(uid, 'quality') == 'good']
print(uids)

print(good_units)

[0, 5, 7, 8, 9, 10, 11, 12, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 29, 30, 31, 33, 34, 37, 38, 40, 41, 43, 44, 45, 47, 50, 51, 52, 55, 68, 118, 119, 120, 121]
[22, 55, 118, 119, 120, 121]


In [170]:
# validation = st.validation.compute_metrics(
#     recording=rec,
#     sorting=sor,
#     verbose=True,
#     return_dict=True,
#     metric_names=['num_spikes', 'firing_rate']
# )
# 
res = st.postprocessing.compute_waveform_features(
    sorting=sor,
    recording=rec,
    as_dataframe=True,
    save_property_or_features=True,
    upsampling_factor=4,
)



c:\users\thijs\miniconda3\envs\spiketoolkit\lib\site-packages\scipy\stats\_stats_mstats_common.py:130: RuntimeWarning:

invalid value encountered in double_scalars

c:\users\thijs\miniconda3\envs\spiketoolkit\lib\site-packages\scipy\stats\_stats_mstats_common.py:140: RuntimeWarning:

invalid value encountered in sqrt

c:\users\thijs\miniconda3\envs\spiketoolkit\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning:

invalid value encountered in greater

c:\users\thijs\miniconda3\envs\spiketoolkit\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning:

invalid value encountered in less

c:\users\thijs\miniconda3\envs\spiketoolkit\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning:

invalid value encountered in less_equal

c:\users\thijs\miniconda3\envs\spiketoolkit\lib\site-packages\scipy\stats\_stats_mstats_common.py:142: RuntimeWarning:

invalid value encountered in double_scalars



In [225]:
from spiketoolkit.postprocessing.plot_features import plot_metrics

for uid in good_units:
    plot_metrics(
        sor,
        rec,
        uid,
        invert_template=True,
        upsampling_factor=4,

    )
# plot_metrics(
#     sor,
#     rec,
#     good_units[0],
#     invert_template=True,
#     upsampling_factor=4,
#
# )

In [141]:
from scipy.signal import resample
import numpy as np
import spiketoolkit.postprocessing.waveform_features_library as wfl
unit_id = good_units[0]
unit_props = sor.get_unit_property_names(unit_id)
max_chan = st.postprocessing.get_unit_max_channels(
    sorting=sor,
    recording=rec,
    unit_ids=unit_id
)[0]
template = sor.get_unit_property(unit_id, 'template')
n_up = template.shape[1] * us
fs = sor.get_sampling_frequency() * us
template = resample(template, n_up, axis=1)

time = np.arange(0, template.shape[1]) * (1/fs) * 1000  # in ms
time = resample(time, n_up)

p0, p1 = wfl._get_trough_and_peak_idx(waveform=-template, fs=fs, max_time_after_trough=1)
print(time.shape, template.shape)
print(time[0], time[-1])

508
508
507
510
(768,) (4, 768)
0.0 5.9921875
[381 381 380 383] [484 505 482 472]


In [210]:

res[good_units[0]].recovery_slope/1000

0    24.039567
1     8.741076
2    28.328622
3    14.709908
Name: recovery_slope, dtype: float64